In [1]:
import re
import hanja # !pip install hanja
import pandas as pd

train_dataset = pd.read_csv('../../../data/train.csv')
train_texts = list(train_dataset['text'].values)
IDs = list(train_dataset['ID'].values)

korean = re.compile('[^가-힣...…\s]+') # 한국어, ..., …, 공백 외의 문자가 한 번 이상 등장할 경우에 대한 패턴

train_dataset['polluted_lv'] = None
train_dataset['no_hanja'] = None
for id_ in IDs:
    text = train_dataset[train_dataset['ID']==id_]['text'].values[0]
    text = hanja.translate(text, 'substitution')
    text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", text)
    
    results = korean.findall(text)
    total = sum([len(r) for r in results])
    prob = total / len(text)
    train_dataset.loc[train_dataset['ID']==id_, 'polluted_lv'] = prob
    train_dataset.loc[train_dataset['ID']==id_, 'no_hanja'] = text

# train_dataset.to_csv('train-no_special-no_hanja-polluted_lv.csv')

In [307]:
# train = pd.read_csv('train-no_special-no_hanja-polluted_lv.csv').drop(columns=['Unnamed: 0'])
train_dataset[(train_dataset['target']==0) & (0.30<train_dataset['polluted_lv']) & (train_dataset['polluted_lv']<0.35)].sample(5)

,ID,text,target,polluted_lv,no_hanja
132,ynat-v1_train_00132,"올해iEIDFO*상에 멀리 개 짖는 소w가 들""S",0,0.32,올해iEIDFO상에 멀리 개 짖는 소w가 들S
717,ynat-v1_train_00717,기중기 \트B들0 기#7*7]y 세계K최 X용,0,0.304348,기중기 트B들0 기77y 세계K최 X용
1240,ynat-v1_train_01240,"날씨-투표% ""는 ~l비 T다 오fm터8G쳐JY일",0,0.347826,날씨투표 는 l비 T다 오fm터8G쳐JY일
2723,ynat-v1_train_02723,"8간Z""로벌 트렌드 n035·마르케Q의%서재에서",0,0.304348,8간Z로벌 트렌드 n035마르케Q의서재에서
215,ynat-v1_train_00215,"폭""a!4 야o0 보러 x려든z시민xM+종*화I관6전시 인기",0,0.344828,폭a4 야o0 보러 x려든z시민xM종화I관6전시 인기


### LLaMA 불러오기

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B' # 'beomi/Llama-3-Open-Ko-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto'
)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): L

### 키워드 추출

In [293]:
PROMPT = '''당신은 기사 제목을 보고 어떤 분야의 기사인지 맞추는 전문가입니다.
[지시사항]
1. 주어진 데이터는 개행 기호(\\n)로 구분된 뉴스 기사 제목들입니다.
2. 주어진 데이터에는 임의의 자리에 문맥에 맞지 않는 글자가 무작위로 삽입되어 있습니다.
3. 해당 데이터들을 가장 잘 표현하는 포괄적인 기사 분야를 한 단어로만 출력하세요.'''

fewshot = [
    {"role": "user", "content": "신간2문학A음악이 q야기ji"},
    {"role": "assistant", "content": "문화"},
    {"role": "user", "content": "정부 4월 한반도 위설d근거O다Y되J Q6야d합"},
    {"role": "assistant", "content": "정치"},
    {"role": "user", "content": "충북교육청 노h조합mYfR동K군별 4양한 목소OH용"},
    {"role": "assistant", "content": "사회"},
    {"role": "user", "content": "국제k호단체HrUTs 유엔 테단체 1정 움직f에k우려"},
    {"role": "assistant", "content": "해외"}
]

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

keywords = []
for i in range(7):
    keys = []
    for j in range(10):
        data = train_dataset[(train_dataset['target']==i) & (0.30<train_dataset['polluted_lv']) & (train_dataset['polluted_lv']<0.40)].sample(5)['no_hanja'].values
        data = '\n'.join(data)

        messages = [{"role": "system", "content": f"{PROMPT}"}] + fewshot + [{"role": "user", "content": f"{data}"}]

        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors='pt'
        ).to(model.device)

        outputs = model.generate(
            input_ids,
            max_new_tokens=1024,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )
        keys.append(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

    keywords.append(keys)

keywords

[['사회', '예술', '경제', '신문', '사회', '문화', '사회', '스포츠', '문화', '자동차'],
 ['스포츠', '스포츠', '스포츠', '스포츠', '스포츠', '스포츠', '스포츠', '스포츠', '스포츠', '스포츠'],
 ['정치', '정치', '정치', '정치', '정치', '정치', 'techno', '경제', '정치', '정치'],
 ['교육', '경제', '정치', '경제', '교육', '기술', '정치', '경제', '교육', '정치'],
 ['IT/기술', '기술', '기술', '기술', '기술', ' 기술', ' 기술', '기술', '디지털', '기술'],
 ['경제', '기술', '기술', '경제', '경제', '경제', '경제', '경제', '경제', 'เทคโนโลย'],
 ['외교', '외교', '경제', '정치', '정치', '외교', '국제', '외교', '정치', '외교']]

In [298]:
PROMPT = '''제공된 데이터들은 뉴스 기사 분야 10개에 대한 단어입니다. 키워드를 추출하세요.
- 많이 등장하는 단어보다 전체 단어들의 맥락을 고려하세요.
- 여러 개의 키워드가 있다면 가장 포괄적인 키워드 한 개만 출력하세요.'''

# fewshot = [
#     {"role": "user", "content": "경제, 문화, 예술, 과학, 문화, 예술, 경제"},
#     {"role": "assistant", "content": "생활"}
# ]

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

final_keys = []
for keys in keywords:
    keys = ', '.join(keys)

    messages = [{"role": "system", "content": f"{PROMPT}"}] + fewshot + [{"role": "user", "content": f"{keys}"}]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors='pt'
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    final_keys.append(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

final_keys

['사회', '스포츠', '정치', '교육, 정치, 경제', '디지털', '경제', '외교']

In [93]:
import json

key_maps = {}
for idx, key in enumerate(final_keys):
    key_maps[idx] = key

with open('./key_maps.json', 'w') as f:
    json.dump(key_maps, f, ensure_ascii=False)